In [16]:
import os
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader
from langchain.chains import RetrievalQA
from dotenv import load_dotenv

Loading the gemini key

In [17]:
load_dotenv()
api_key = os.getenv("GOOGLE_API_KEY") #make sure .env file and place your gemini key

if not api_key:
    raise ValueError("GOOGLE_API_KEY not found in .env file.")

os.environ["GOOGLE_API_KEY"] = api_key
print("Gemini API key loaded")


Gemini API key loaded


PDF Chunk creating and loading

In [18]:
def load_pdf(file_path):
    loader = PyPDFLoader(file_path)
    pages = loader.load()
    
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=500,
        chunk_overlap=100
    )
    
    chunks = splitter.split_documents(pages)
    return chunks

pdf_path = "RAG_Agent.pdf" #Path
documents = load_pdf(pdf_path)
print(f"Loaded and split into {len(documents)} chunks.")


Loaded and split into 8 chunks.


Embedding and Vector Store

In [19]:
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
vector_db = FAISS.from_documents(documents, embeddings)
print("vector store created.")


vector store created.


Prompt Template

In [20]:
from langchain.prompts import PromptTemplate
custom_prompt = PromptTemplate(
    input_variables=["summaries", "question"],
    template="""
You are a helpful assistant that only answers based on the provided context.
Do not use prior knowledge.

Context:
{summaries}

Question:
{question}

Only answer from the context. If the answer is not in the context, say "I don't know."
"""
)



LLM Creation

In [21]:
llm = ChatGoogleGenerativeAI(
    model="models/gemini-1.5-flash",  
    temperature=0.3,
    max_output_tokens=1024,
    top_p=0.8,
    top_k=40
)

In [22]:
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory

# Create memory object
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True,
    output_key="answer"
)

Retreiver

In [23]:
qa_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=vector_db.as_retriever(),
    memory=memory,
    return_source_documents=True,
    condense_question_prompt=custom_prompt
)


print("QA chain ready.")

QA chain ready.


Q & A

In [ ]:
# question = "Yes i have reached on the main gate ,now where?"

# response = qa_chain({"question": question})

# print("AI Answer:", response["answer"])  # use "answer" instead of "result"


# # Show relevant source context
# # for i, doc in enumerate(response["source_documents"]):
# #     print(f"\n📄 Source {i+1}:\n{doc.page_content}")
while True:
    question = input("You: ")

    if question.lower() in ["exit", "quit", "q"]:
        print("🛑 Exiting chat.")
        break

    try:
        response = qa_chain({"question": question})
        print("AI:", response["answer"])
    except Exception as e:
        print("❌ Error:", e)



ValueError: Missing some input keys: {'summaries'}